## Oxford data: government response to covid-19

In [1]:
#load packages
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
import numpy as np
from urllib.request import urlopen
import json

In [2]:
#Load the data
from nepal.datasets import GovernmentResponse
data_gov = GovernmentResponse().load()

This dataset has information on the government policies in 187 countries. We are only interested in the USA, 

In [3]:
set_countries = set(data_gov["CountryName"])
print(len(set_countries))
print(len(data_gov))

187
299127


In [4]:
data_gov = data_gov[data_gov["CountryName"]== "United States"]
print(len(data_gov))
data_gov.describe

45916


<bound method NDFrame.describe of           CountryName CountryCode RegionName RegionCode Jurisdiction  \
245264  United States         USA       <NA>       <NA>    NAT_TOTAL   
245265  United States         USA       <NA>       <NA>    NAT_TOTAL   
245266  United States         USA       <NA>       <NA>    NAT_TOTAL   
245267  United States         USA       <NA>       <NA>    NAT_TOTAL   
245268  United States         USA       <NA>       <NA>    NAT_TOTAL   
...               ...         ...        ...        ...          ...   
291175  United States         USA    Wyoming      US_WY  STATE_TOTAL   
291176  United States         USA    Wyoming      US_WY  STATE_TOTAL   
291177  United States         USA    Wyoming      US_WY  STATE_TOTAL   
291178  United States         USA    Wyoming      US_WY  STATE_TOTAL   
291179  United States         USA    Wyoming      US_WY  STATE_TOTAL   

             Date  C1_combined_numeric C1_combined  C2_combined_numeric  \
245264 2020-01-01         

The data set has data on the national level and on the states' level. The dataset is therefore split up into two datasets; one at national level and one at state level. 

In [5]:
data_gov_nat = data_gov[data_gov["Jurisdiction"]=="NAT_TOTAL"]
data_gov = data_gov[data_gov["Jurisdiction"]=="STATE_TOTAL"]

### Government response at the national level

First, we check whether there are many missing values and how many observations there are on the national level. There is exactly one observations for each day, starting from January 1st, 2020. 

In [6]:
print(len(data_gov_nat))
data_gov_nat.isnull().sum()
data_gov_nat.isnull().sum()

883


CountryName                            0
CountryCode                            0
RegionName                           883
RegionCode                           883
Jurisdiction                           0
Date                                   0
C1_combined_numeric                    6
C1_combined                            6
C2_combined_numeric                    6
C2_combined                            6
C3_combined_numeric                    6
C3_combined                            6
C4_combined_numeric                    6
C4_combined                            6
C5_combined_numeric                    6
C5_combined                            6
C6_combined_numeric                    6
C6_combined                            6
C7_combined_numeric                    6
C7_combined                            6
C8_combined_numeric                    6
C8_combined                            6
E1_combined_numeric                    6
E1_combined                            6
E2_combined_nume

In [7]:
from datetime import date
set_dates = set(data_gov_nat["Date"])
def numOfDays(date1, date2):
    return (date2-date1).days
#we need to substract one because the first day is nog included otherwise
print(min(data_gov_nat["Date"]))
print(max(data_gov_nat["Date"]))
len(set_dates) - numOfDays(min(data_gov_nat["Date"]),max(data_gov_nat["Date"])) - 1#zero so there's a measurement on each date (no days were skipped) 

2020-01-01 00:00:00
2022-06-01 00:00:00


0

The figure below shows the evolution of all containment and closure policies at a national level, in the US, over time. The drop-down button allows to select the policy of interest for a clearer view. Many containment and closure measures have decreased over time and especially since the beginning of January 2021. The only exceptions are travel restrictions and (C8) and Restrictions on gatherings (C4), although the latter has dropped considerably in the last few months.

In [8]:
#Code inspired by this kaggel example https://www.kaggle.com/code/jrmistry/plotly-how-to-change-plot-data-using-dropdowns/notebook

import plotly.graph_objects as go
# Create figure
fig = go.Figure()

for column in ["C1_combined_numeric", "C2_combined_numeric", "C3_combined_numeric", "C4_combined_numeric", "C5_combined_numeric", "C6_combined_numeric", "C7_combined_numeric", "C8_combined_numeric"]:
    fig.add_trace(
        go.Scatter(
            x = data_gov_nat["Date"],
            y = data_gov_nat[column],
            name = column
        )
    )

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
             dict(label = 'C1 - School Closures',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'C1 - School Closures',
                           'showlegend':False}]),
             dict(label = 'C2- Workplace closing',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False, False, False]},
                          {'title': 'C2- Workplace closing',
                           'showlegend':False}]),
             dict(label = 'C3- Cancel Public Events',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False]},
                          {'title': 'C3- Cancel Public Events',
                           'showlegend':False}]),
             dict(label = 'C4- Restrictions on gatherings',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, False]},
                          {'title': 'C4- Restrictions on gatherings',
                           'showlegend':False}]),
             dict(label = 'C5- Public Transportation',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False]},
                          {'title': 'C5- Public Transportation',
                           'showlegend':False}]),
             dict(label = 'C6- Stay at Home Order',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False]},
                          {'title': 'C6- Stay at Home Order',
                           'showlegend':False}]),
             dict(label = 'C7- Restrictions on Internal Movement',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False]},
                          {'title': 'C7- Restrictions on Internal Movement',
                           'showlegend':False}]),
             dict(label = 'C8- International Travel Controls',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False]},
                          {'title': 'C8- International Travel Controls',
                           'showlegend':False}])
            ])
        )
    ])

fig.show()


The figure below shows the evolution of all economic policies at a national level, in the US, over time. The drop-down button allows to select the policy of interest for a clearer view. Both policies were enforced in april 2020 and were finally cancelled in October 2021.

In [9]:
import plotly.graph_objects as go
# Create figure
fig = go.Figure()

for column in ["E1_combined_numeric", "E2_combined_numeric"]:
    fig.add_trace(
        go.Scatter(
            x = data_gov_nat["Date"],
            y = data_gov_nat[column],
            name = column
        )
    )

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
             dict(label = 'E1-Income support',
                  method = 'update',
                  args = [{'visible': [True, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'E1-Income support',
                           'showlegend':False}]),
             dict(label = 'E2-Debt/contract relief',
                  method = 'update',
                  args = [{'visible': [False, True]},
                          {'title': 'E2-Debt/contract relief',
                           'showlegend':False}])
            ])
        )
    ])

fig.show()


The figure below shows the evolution of all health system policies at a national level, in the US, over time. The drop-down button allows to select the policy of interest for a clearer view. It can be seen that facial covering have risen higher and higher until september 2020. Starting May 2021, facials coverage has started a steady decline. Contact tracing is low and stable in the US. Vaccination is the only policy that has steadily increased over time.

In [10]:
import plotly.graph_objects as go
# Create figure
fig = go.Figure()

for column in ["H1_combined_numeric", "H2_combined_numeric", "H3_combined_numeric", "H6_combined_numeric", "H7_combined_numeric", "H8_combined_numeric"]:
    fig.add_trace(
        go.Scatter(
            x = data_gov_nat["Date"],
            y = data_gov_nat[column],
            name = column
        )
    )

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
             dict(label = 'H1-Public information campaigns',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'H1-Public information campaigns',
                           'showlegend':False}]),
             dict(label = 'H2 - Testing Policy',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False]},
                          {'title': 'H2 - Testing Policy',
                           'showlegend':False}]),
             dict(label = 'H3 - Contact tracing',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False]},
                          {'title': 'H3 - Contact tracing',
                           'showlegend':False}]),
             dict(label = 'H6 - Facial coverings',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False]},
                          {'title': 'H6 - Facial coverings',
                           'showlegend':False}]),
             dict(label = 'H7 Vaccination policy',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False]},
                          {'title': 'H7 Vaccination policy',
                           'showlegend':False}]),
             dict(label = 'H8 Protection of elderly people',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False]},
                          {'title': 'H8 Protection of elderly people',
                           'showlegend':False}])
            ])
        )
    ])

fig.show()

The figure below shows the evolution of all vaccination policies at a national level, in the US, over time. The drop-down button allows to select the policy of interest for a clearer view. Except for Vaccine prioritisation, all other three measures increase steadily over time.

In [11]:
import plotly.graph_objects as go
# Create figure
fig = go.Figure()

for column in ["V1_combined_numeric", "V2_combined_numeric", "V3_combined_numeric", "V4_combined_numeric"]:
    fig.add_trace(
        go.Scatter(
            x = data_gov_nat["Date"],
            y = data_gov_nat[column],
            name = column
        )
    )

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True]},
                          {'title': 'All',
                           'showlegend':True}]),
             dict(label = 'V1- Vaccine prioritisation',
                  method = 'update',
                  args = [{'visible': [True, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': 'V1- Vaccine prioritisation',
                           'showlegend':False}]),
             dict(label = 'V2- Vaccine eligibility/availability',
                  method = 'update',
                  args = [{'visible': [False, True, False, False]},
                          {'title': 'V2- Vaccine eligibility/availability',
                           'showlegend':False}]),
             dict(label = 'V3- Vaccine financial support',
                  method = 'update',
                  args = [{'visible': [False, False, True, False]},
                          {'title': 'V3- Vaccine financial support',
                           'showlegend':False}]),
             dict(label = 'V4- Mandatory vaccination',
                  method = 'update',
                  args = [{'visible': [False, False, False, True]},
                          {'title': 'V4- Mandatory vaccination',
                           'showlegend':False}])
            ])
        )
    ])

fig.show()

### Government response at the state level

This section describes the government response at the state level. We will show a map with the evolution of the following four indeces over time:

* Government response index
* Containment and health index
* Stringency index
* Economic support index


In [ ]:
# make dataset
data_gov["State"] = data_gov["RegionCode"].map(lambda x: x.lstrip('US_'))
data_gov["EconomicSupportIndexForDisplay"]
data_gov["dow"] = data_gov["Date"].dt.dayofweek
data_gov_sample = data_gov[data_gov["dow"]==1] # only keep weekly updates
data_gov_sample["Date"] = pd.to_datetime(data_gov_sample["Date"])
data_gov_sample = data_gov_sample.sort_values('Date', ascending=True)
data_gov_sample['Date'] = data_gov_sample['Date'].dt.strftime('%m-%d-%Y')
with urlopen(
    "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
) as response:
    counties = json.load(response)

### Government response index

In [51]:
print(min(data_gov_sample["GovernmentResponseIndexForDisplay"]))
print(max(data_gov_sample["GovernmentResponseIndexForDisplay"]))
print(data_gov_sample["GovernmentResponseIndexForDisplay"].mean() )
print(data_gov_sample["GovernmentResponseIndexForDisplay"].median())

0.0
80.21
49.71397529299969
51.04


In [ ]:
fig = px.choropleth(data_gov_sample, geojson=counties, 
                    locations=data_gov_sample['State'], 
                    locationmode="USA-states", 
                    color='GovernmentResponseIndexForDisplay',
                    range_color=(0,100),
                    scope="usa",
                    animation_frame="Date",
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Containment and health index

In [54]:
print(min(data_gov_sample["ContainmentHealthIndexForDisplay"]))
print(max(data_gov_sample["ContainmentHealthIndexForDisplay"]))
print(data_gov_sample["ContainmentHealthIndexForDisplay"].mean() )
print(data_gov_sample["ContainmentHealthIndexForDisplay"].median())

0.0
80.48
50.92833386126069
52.38


In [ ]:
fig = px.choropleth(data_gov_sample, geojson=counties, 
                    locations=data_gov_sample['State'], 
                    locationmode="USA-states", 
                    color='ContainmentHealthIndexForDisplay',
                    range_color=(0,100),
                    scope="usa",
                    animation_frame="Date",
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Stringency index

In [50]:
print(min(data_gov_sample["StringencyIndexForDisplay"]))
print(max(data_gov_sample["StringencyIndexForDisplay"]))
print(data_gov_sample["StringencyIndexForDisplay"].mean() )
print(data_gov_sample["StringencyIndexForDisplay"].median())

0.0
93.52
44.37249247345905
42.59


In [ ]:
fig = px.choropleth(data_gov_sample, geojson=counties, 
                    locations=data_gov_sample['State'], 
                    locationmode="USA-states", 
                    color='StringencyIndexForDisplay',
                    range_color=(0,100),
                    scope="usa",
                    animation_frame="Date",
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Economic support index

In [43]:
print(min(data_gov_sample["EconomicSupportIndexForDisplay"]))
print(max(data_gov_sample["EconomicSupportIndexForDisplay"]))
print(data_gov_sample["EconomicSupportIndexForDisplay"].mean() )
print(data_gov_sample["EconomicSupportIndexForDisplay"].median())

0.0
100.0
41.2745953665503
37.5


In [ ]:
with urlopen(
    "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
) as response:
    counties = json.load(response)

fig = px.choropleth(data_gov_sample, geojson=counties, 
                    locations=data_gov_sample['State'], 
                    locationmode="USA-states", 
                    color='EconomicSupportIndexForDisplay',
                    range_color=(0,100),
                    scope="usa",
                    animation_frame="Date",
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()